In [123]:
import sqlite3
import pandas as pd

In [124]:
# Conecta ao banco (pode ser em memória ou arquivo .db)
conn = sqlite3.connect("empresa.db")
cur = conn.cursor()

In [125]:
# Remove tabela anterior, se já existir
cur.execute("DROP TABLE IF EXISTS colaboradores")

# Cria a tabela
cur.execute("""
CREATE TABLE colaboradores (
    id INTEGER NOT NULL,
    nome VARCHAR NOT NULL,
    salario INTEGER NOT NULL,
    lider_id INTEGER,
    UNIQUE(id)
)
""")

# Insere os dados
dados = [
    (40, 'Helen', 1500, 50),
    (50, 'Bruno', 3000, 10),
    (10, 'Leonardo', 4500, 20),
    (20, 'Marcos', 10000, None),
    (70, 'Mateus', 1500, 10),
    (60, 'Cinthia', 2000, 70),
    (30, 'Wilian', 1501, 50)
]

cur.executemany("INSERT INTO colaboradores VALUES (?, ?, ?, ?)", dados)
conn.commit()

# Consulta para conferir os dados
df = pd.read_sql_query("SELECT * FROM colaboradores", conn)
print(df)


   id      nome  salario  lider_id
0  40     Helen     1500      50.0
1  50     Bruno     3000      10.0
2  10  Leonardo     4500      20.0
3  20    Marcos    10000       NaN
4  70    Mateus     1500      10.0
5  60   Cinthia     2000      70.0
6  30    Wilian     1501      50.0


In [126]:
df = pd.read_sql_query("""
SELECT 
    id, 
    nome, 
    salario, 
    lider_id
FROM colaboradores
""", conn)

print(df)

   id      nome  salario  lider_id
0  40     Helen     1500      50.0
1  50     Bruno     3000      10.0
2  10  Leonardo     4500      20.0
3  20    Marcos    10000       NaN
4  70    Mateus     1500      10.0
5  60   Cinthia     2000      70.0
6  30    Wilian     1501      50.0


In [241]:
query = """
    SELECT 
        id AS funcionario_id,
        lider_id AS chefe_id,
        0 as nivel
    FROM colaboradores
"""

df = pd.read_sql_query(query, conn)
print(df)


   funcionario_id  chefe_id  nivel
0              40      50.0      0
1              50      10.0      0
2              10      20.0      0
3              20       NaN      0
4              70      10.0      0
5              60      70.0      0
6              30      50.0      0


In [242]:
query = """
WITH RECURSIVE Hierarquia AS (
    SELECT id AS funcionario_id, lider_id AS chefe_id, 0 AS nivel
    FROM colaboradores

    UNION ALL
    SELECT h.funcionario_id, c.lider_id AS chefe_id, h.nivel + 1
    FROM Hierarquia h
    JOIN colaboradores c ON h.chefe_id = c.id
    WHERE h.chefe_id IS NOT NULL
)
SELECT * FROM Hierarquia
order BY funcionario_id, nivel;
"""

df = pd.read_sql_query(query, conn)
print(df)

    funcionario_id  chefe_id  nivel
0               10      20.0      0
1               10       NaN      1
2               20       NaN      0
3               30      50.0      0
4               30      10.0      1
5               30      20.0      2
6               30       NaN      3
7               40      50.0      0
8               40      10.0      1
9               40      20.0      2
10              40       NaN      3
11              50      10.0      0
12              50      20.0      1
13              50       NaN      2
14              60      70.0      0
15              60      10.0      1
16              60      20.0      2
17              60       NaN      3
18              70      10.0      0
19              70      20.0      1
20              70       NaN      2


In [243]:
query = """
WITH RECURSIVE Hierarquia AS (
    SELECT id AS funcionario_id, lider_id AS chefe_id, 0 AS nivel
    FROM colaboradores
    UNION ALL
    SELECT h.funcionario_id, c.lider_id AS chefe_id, h.nivel + 1
    FROM Hierarquia h
    JOIN colaboradores c ON h.chefe_id = c.id
    WHERE h.chefe_id IS NOT NULL
)
SELECT 
  h.funcionario_id, 
  h.chefe_id,
  f.salario AS salario_func, 
  ch.salario AS salario_chefe,
  h.nivel
FROM Hierarquia h
JOIN colaboradores f ON f.id = h.funcionario_id
JOIN colaboradores ch ON ch.id = h.chefe_id
WHERE ch.salario >= 2 * f.salario;
"""

df = pd.read_sql_query(query, conn)
print(df)

    funcionario_id  chefe_id  salario_func  salario_chefe  nivel
0               40        50          1500           3000      0
1               10        20          4500          10000      0
2               70        10          1500           4500      0
3               40        10          1500           4500      1
4               50        20          3000          10000      1
5               70        20          1500          10000      1
6               60        10          2000           4500      1
7               30        10          1501           4500      1
8               40        20          1500          10000      2
9               60        20          2000          10000      2
10              30        20          1501          10000      2


In [244]:
query = """
WITH RECURSIVE Hierarquia AS (
    -- mesma recursiva
    SELECT id AS funcionario_id, lider_id AS chefe_id, 0 AS nivel
    FROM colaboradores
    UNION ALL
    SELECT h.funcionario_id, c.lider_id AS chefe_id, h.nivel + 1
    FROM Hierarquia h
    JOIN colaboradores c ON h.chefe_id = c.id
    WHERE h.chefe_id IS NOT NULL
),
ChefeSalario AS (
    SELECT 
      h.funcionario_id, 
      h.chefe_id,
      h.nivel
    FROM Hierarquia h
    JOIN colaboradores f ON f.id = h.funcionario_id
    JOIN colaboradores ch ON ch.id = h.chefe_id
    WHERE ch.salario >= 2 * f.salario
),
MenorChefe AS (
    SELECT funcionario_id, chefe_id
    FROM ChefeSalario
    WHERE (funcionario_id, nivel) IN (
        SELECT funcionario_id, MIN(nivel) FROM ChefeSalario GROUP BY funcionario_id
    )
)
SELECT 
    c.id AS funcionario_id, 
    mc.chefe_id
FROM colaboradores c
LEFT JOIN MenorChefe mc ON c.id = mc.funcionario_id
ORDER BY c.id;
"""

df = pd.read_sql_query(query, conn)
print(df)

   funcionario_id  chefe_id
0              10      20.0
1              20       NaN
2              30      10.0
3              40      50.0
4              50      20.0
5              60      10.0
6              70      10.0
